# Perceptron Multicamada no problema MNIST



### Disciplina: Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte e Matheus das Neves Fernandes

## Introdução


Este trabalho implementa um Perceptron Multicamada (MLP) para resolver o problema de classificação de dígitos manuscritos do conjunto de dados MNIST. Foram exploradas diferentes arquiteturas e técnicas de treinamento, avaliando o desempenho de cada abordagem.


## 1. Preparação do Ambiente
Nesta seção, importamos as bibliotecas necessárias, carregamos o conjunto de dados MNIST e preparamos o dataloader


### 1.1. Importação de Bibliotecas

In [34]:
import tensorflow as tf
from d2l import tensorflow as d2l
d2l.use_svg_display()
import pdb
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

### 1.2. Carregamento e Pré-processamento dos Dados

In [35]:
class MNIST(d2l.DataModule):  # @save
    """O dataset MNIST."""
    
    def __init__(self, batch_size=64):
        super().__init__()
        self.save_hyperparameters()
        self.train, self.val = tf.keras.datasets.mnist.load_data()
        
    def get_dataloader(self, train):
        data = self.train if train else self.val
        process = lambda X, y: (tf.expand_dims(X, axis=3) / 255,tf.cast(y, dtype='int32'))
        resize_fn = lambda X, y: (tf.image.resize_with_pad(X,*self.resize), y)
        shuffle_buf = len(data[0]) if train else 1
        return tf.data.Dataset.from_tensor_slices(process(*data)).batch(self.batch_size).shuffle(shuffle_buf)

#Instancia o dataset
data = MNIST()

In [36]:
# Verifica o número de exemplos e formato dos dados
print(len(data.train[0]), len(data.val[0]))
print(data.train[0].shape)

60000 10000
(60000, 28, 28)


In [37]:
#Verifica um minibatch
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

(64, 28, 28, 1) <dtype: 'float32'> (64,) <dtype: 'int32'>


## 2. Modelos e Arquiteturas
Nesta seção, implementamos diferentes arquiteturas e técnicas de treinamento para o MLP, avaliando o impacto no desempenho.


### 2a. Perceptron com uma Camada Escondida, Função Logística, Custo SSE e Descida de Gradiente


In [38]:
# Converte os rótulos para One-Hot
def preprocess_labels(dataset):
    return dataset.map(lambda X, y: (X, tf.one_hot(y, depth=10)))

# Prepara os dados para treinamento com os rotulos processados
train_dataloader = preprocess_labels(data.get_dataloader(train=True))
val_dataloader = preprocess_labels(data.get_dataloader(train=False))

In [39]:
model = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  #Transforma a entrada (28x28) em vetor (784)
    tf.keras.layers.Dense(128, activation='sigmoid'),   #Camada escondida
    tf.keras.layers.Dense(10, activation='sigmoid')  #Camada de saída
])   

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate = 0.1),   #Descida por gradiente
    loss=tf.keras.losses.MeanSquaredError(),  #Erro quadrático médio
    metrics=['accuracy']
)

model.fit(
    train_dataloader,               # Dados de treinamento
    epochs=10,                       # Número de épocas
    validation_data=val_dataloader  # Dados de validação
)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.2086 - loss: 0.1028 - val_accuracy: 0.3573 - val_loss: 0.0841
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.4097 - loss: 0.0820 - val_accuracy: 0.5618 - val_loss: 0.0749
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5874 - loss: 0.0728 - val_accuracy: 0.6709 - val_loss: 0.0653
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6846 - loss: 0.0635 - val_accuracy: 0.7332 - val_loss: 0.0571
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7273 - loss: 0.0560 - val_accuracy: 0.7748 - val_loss: 0.0507
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7673 - loss: 0.0499 - val_accuracy: 0.7954 - val_loss: 0.0460
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7885 - loss: 0.0454 - val_accuracy: 0.8161 - val_loss: 0.0423
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8066 - loss: 0.0423 - val_accuracy: 0.

### 2b. Saída do tipo softmax, custo "entropia cruzada"

In [40]:
# Prepara os dados de treinamento sem conversão para One-Hot
train_dataloader = data.get_dataloader(train=True)
val_dataloader = data.get_dataloader(train=False)

In [41]:
model = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  #Transforma a entrada (28x28) em vetor (784)
    tf.keras.layers.Dense(128, activation='sigmoid'),   #Camada escondida
    tf.keras.layers.Dense(10, activation='softmax')  #Camada de saída
])   

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate = 0.1),   #Descida por gradiente
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

model.fit(
    train_dataloader,               # Dados de treinamento
    epochs=10,                      # Número de épocas
    validation_data=val_dataloader  # Dados de validação
)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7253 - loss: 1.1301 - val_accuracy: 0.8986 - val_loss: 0.3755
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8975 - loss: 0.3717 - val_accuracy: 0.9135 - val_loss: 0.3106
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9094 - loss: 0.3169 - val_accuracy: 0.9173 - val_loss: 0.2844
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9188 - loss: 0.2890 - val_accuracy: 0.9226 - val_loss: 0.2643
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9250 - loss: 0.2607 - val_accuracy: 0.9294 - val_loss: 0.2470
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9300 - loss: 0.2465 - val_accuracy: 0.9317 - val_loss: 0.2355
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9329 - loss: 0.2312 - val_accuracy: 0.9359 - val_loss: 0.2239
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9365 - loss: 0.2243 - val_accuracy: 0.